# Sandbox

In [2]:
# For type annotations
from typing import Callable

from docplex.mp.dvar import Var
from docplex.mp.model import Model
from docplex.mp.constr import LinearConstraint

In [3]:
def _find_root(func: Callable[[float], float], interval: tuple[float, float], precision: float) -> float:
    (a, b) = interval
    interval_length = b - a

    if func(a) * func(b) >= 0:
        raise Exception('No root guaranteed to exist in the interval ' + str(interval))

    mean = (a + b) / 2

    if precision >= interval_length:
        return mean

    if func(mean) == 0:
        return mean

    if func(mean) * func(b) < 0:
        new_interval = (mean, b)

    if func(a) * func(mean) < 0:
        new_interval = (a, mean)

    return _find_root(func, new_interval, precision)

In [4]:

def stretch_to_the_right(model: Model, constraint: LinearConstraint, precision: float = 0.0):
	def f(value_to_try: float) -> float:
		constraint.right_expr.constant = value_to_try
		model.solve()
		if model.solution is None:
			return 1.0
		else:
			return -1.0

	x: Var = constraint.left_expr
	interval = (constraint.right_expr.constant, x.ub)

	# constraint.right_expr.constant = x.ub
	model.maximize(x)
	# model.solve()

	# constraint.right_expr.constant = x.solution_value

	_find_root(f, interval, precision)

	model.remove_objective()

In [41]:
# def test_base_on_another_constraint(self):
model = Model()
x = model.continuous_var(lb=0, ub=5, name='x')
model.add_constraint(x <= 3)
constraint = model.add_constraint(x <= 0, ctname='constraint to stretch')

# precision = 0.01
stretch_to_the_right(model, constraint)

assert constraint.right_expr.constant == 3

Exception: No root guaranteed to exist in the interval (0, 5)

In [30]:
m = Model('sandbox')
x = m.continuous_var(0, 20, 'x')

In [31]:
m.maximize(x)

In [32]:
c = m.add_constraint(x <= 10)

In [39]:
c.rhs.constant = 18

In [40]:
m.solve()
x.solution_value

18.0